In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [8]:
%pip install -U openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)

   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   ---------------------------------------- 2/2 [openpyxl]

Note: you may need to restart the kernel to use updated packages.


In [9]:

# 복사본 작업
# df = commerce_df.copy()
df = pd.read_excel('datatata/E_Commerce_Dataset.xlsx',sheet_name=1)

# 1. 결축치 처리
df['Tenure'] = df['Tenure'].fillna(df['Tenure'].median())
df['WarehouseToHome'] = df['WarehouseToHome'].fillna(df['WarehouseToHome'].median())
df['HourSpendOnApp'] = df['HourSpendOnApp'].fillna(df['HourSpendOnApp'].median())

df['NoLastYearPurchase'] = df['OrderAmountHikeFromlastYear'].isna().astype(int)
df['OrderAmountHikeFromlastYear'] = df['OrderAmountHikeFromlastYear'].fillna(0)

df['CouponUsed'] = df['CouponUsed'].fillna(0)
df['OrderCount'] = df['OrderCount'].fillna(0)

# NeverOrdered로는 구분하고,
# DaySinceLastOrder는 명확히 큰 값 (ex. 최대값 + 1) 으로 채워서 모델이 구분 가능하게끔 설정
max_day = df['DaySinceLastOrder'].max()
df['DaySinceLastOrder'] = df['DaySinceLastOrder'].fillna(max_day + 1)
df['NeverOrdered'] = (df['DaySinceLastOrder'] > max_day).astype(int)

# 2. 범주형 인코딩
cat_cols = ['PreferredLoginDevice', 'PreferredPaymentMode', 'Gender',
            'PreferedOrderCat', 'MaritalStatus']

# 나중에 새로운 데이터를 인코딩할때 사용하기 위해서 저장해두기.
le_dict = {}
for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    le_dict[col] = le

# 3. ID 제거
df.drop(columns=['CustomerID'], inplace=True)


In [10]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

# 복사본 생성
df1 = df.copy()
df2 = df.copy()
df3 = df.copy()

# 타겟 컬럼 제외한 연속형 수치형 컬럼 추출

# 숫자형 컬럼 중에서도 범주형 의미 없는 것만 선택
exclude = ['Churn', 'NeverOrdered', 'CityTier', 'PreferredPaymentMode',
           'Gender', 'PreferredOrderCat', 'MaritalStatus', 'PreferredLoginDevice']

num_cols = df.select_dtypes(include=['float64', 'int64']).columns.difference(exclude)

# 스케일러 정의
scaler_standard = StandardScaler()
scaler_minmax = MinMaxScaler()
scaler_robust = RobustScaler()

# 스케일링 적용
df1[num_cols] = scaler_standard.fit_transform(df1[num_cols])
df2[num_cols] = scaler_minmax.fit_transform(df2[num_cols])
df3[num_cols] = scaler_robust.fit_transform(df3[num_cols])

In [11]:
df1.to_csv('datatata/finalcsv.csv', index=False)